# L2 Scraping Notebook

L2 data is our second level of unlabeled data. Here, we are collecting articles and webpages on the 7 topics that we are deciding candidates' opinions on. The 7 topics that have been selected are listed below:

1. Guns - Should the US have stricter gun control?
2. Immgiration - Should we open up our borders and grant a path to citizenship for current undocumented immigrants?
3. Healthcare - Should we support Meidcare for all?
4. Abortion - Does the woman have a right to choose to continue with pregnancy?
5. College - Should higher education be free?
6. Military - Should we increase military spending?
7. Taxes - Should we implement a tax on extreme wealth?

Note, while there are other issues that are relavent to the election, the omission of those topics from a platform is more indicative than statements that are actually against that issue. For example, political candidates who do not support the fight against climate change are more likely to not discuss climate change in their platforms rather than say that climate change is a hoax.

In [ ]:
# Import libraries. For scraping, we will be using the BeautifulSoup library

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import glob, os    

headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})

In [ ]:
# Helper function
# INPUT: a list of links to pull text from, the category / topic of the list of links, the partition of data, and a list of keywords (at least one word from the keyword list must match to be included in the result)
# OUTPUT: a set of dataframes (split in batches of 100 links) containing the text from the articles. Each paragraph is in its own row
def get_body_updated_to_df(links, cat, part, rgx):
    texts = []
    matching_links = []
    n = len(links)
    print(n)
    i = 0
    dfn = 0
    for link in links:
        # Try to get the text from the links
        try:
            req = requests.get(link, headers)
            soup = BeautifulSoup(req.content, 'html.parser')
            s = soup.get_text()
            # Split the text into paragraphs
            all_strings = s.split("\n")
            
            # Keep the first 512 words per paragraph. Note paragraphs are typically shorter. 
            # We initally kept this at 512 in case we wanted BERT to pretrain on paragraphs. We ended up splitting the paragraphs into sentences and pretraining on individual sentences
            # Also we removed all strings that were under 50 characters. Shorter sentences would not be helpful to train on
            body_512 = [' '.join(string.split()[:512 ]) for string in all_strings if len(string) > 50]
            texts.extend(body_512)
            
            # Also note the source that the text came from
            matching_links.extend([link] * len(body_512))
        except:
            print("Cannot get text")
        
        i += 1
        
        # Convert result into a dataframe
        # Give a status update
        if i%10 == 0:
            print("Completed ", i, " scrapes out of ", n)
        
        # Incrementally create dataframes in chunks of 100 links
        if i%100 == 0 or i == n - 1:
            
            df = pd.DataFrame(list(zip(matching_links, texts)), columns =['url', 'text']) 
            df["category"] = cat
            df_cleaned = df[df.text.str.contains(rgx)]
            
            # Save as a CSV
            df_cleaned.to_csv(cat + "_" + str(part) + str(dfn) + ".csv")
            dfn += 1
            
            # Reset text and matching links to build a new dataframe
            texts = []
            matching_links = []

            
    return(texts, matching_links)

For all topics, we first start with a set of URLs that are pulled from researching a certain topic. We then pull all the articles that these articles link to. If this is a valid URL and matches a keyword indicating it is related to the topic of interest, we add it to the list of links to pull text from.

Once we have a full list of links, we create dataframes containing the text within these articles

## Steps 1 and 2
Step 1: create a list of links to scrape 

Step 2: pull the text from these links to create our corpus

In [ ]:
# HELPER FUNCTION
# Input: base list of links and accepted words for the URL to be included in the scrape list
# Output: list of scraped links
def get_scraped_links(base_list, accepted_words):
    # Initialize empty list for our scraped links to go into
    scraped_links = []
    
    # Iterate through all URLs in the base list of links to pull all the links that these original links refer to
    # Add these links to our list of scraped linkes
    for url in base_list:
        parent = 'https://' + url.split('/')[2]
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        # For all links that a link in the base_list refers to:
        for link in soup.find_all('a'):
            
            # Pull the url. If it is an internal link, make this a full URL so we can scrape the full text later on
            try:
                l = link.get('href')
                if l[0] == '/': 
                    l = parent + l
            
                # Ensure the link URL is related by seeing if the url contains at least one of the accepted words
                if any(w in l for w in accepted_words):
                    scraped_links.append(l)
            except:
                print('no url')

    # Remove duplicates
    scraped_links = list(set(scraped_links))
    
    # Print the number of links that have been scraped
    print(len(scraped_links))
    
    return scraped_links

In [ ]:
# Gun Control 

all_gun_links = ["https://www.nytimes.com/topic/subject/guns-and-gun-control"
                 ,"https://abcnews.go.com/alerts/gun-control",
                 "https://theconversation.com/us/topics/us-gun-control-21136",
                 "https://theconversation.com/us/topics/us-gun-control-21136?page=2",
                 "https://theconversation.com/us/topics/us-gun-control-21136?page=3", 
                 "https://time.com/tag/gun-control/", 
                 "https://time.com/tag/gun-control/?page=2", 
                 "https://www.thenation.com/subject/guns-and-gun-control/",
                 "https://www.npr.org/tags/132845450/gun-control", 
                 "https://www.commondreams.org/tag/gun-control", 
                 "https://www.nbcnews.com/guns-in-america", 
                 "https://www.texasmonthly.com/tag/gun-control/",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk00ZoIRZaon36BRCcASEZta6PTC4bQ%3A1582179625637&ei=KSVOXt-vJpPy-gSnsJX4Bg&q=gun+control+debate&oq=gun+control+debate&gs_l=psy-ab.3..0l10.4278.5823..6094...1.7..0.78.473.7....2..0....1..gws-wiz.......0i71j0i273i395j0i395j0i273i395i70i249j0i22i30j0i273.m77ghAJfkX8&ved=0ahUKEwif6Lm0vt_nAhUTuZ4KHSdYBW8Q4dUDCAs&uact=5",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk01nFnPWvojfpbj_JrjuoslX_nYXaQ:1582179632637&q=gun+control+debate+articles&sa=X&ved=2ahUKEwislOW3vt_nAhXIo54KHTt4ABIQ1QIoAHoECA8QAQ&biw=1766&bih=767",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk02hEYJhmptFnDs9NBewlbfKWDYwPA:1582179657560&q=gun+control+issues&sa=X&ved=2ahUKEwi8pNbDvt_nAhUXv54KHfC_B0cQ1QIoAnoECAwQAw&biw=1766&bih=767", 
                 "https://www.google.com/search?q=anti+gun+control+articles&rlz=1C5CHFA_enUS811US824&oq=anti+gun+control+articles&aqs=chrome..69i57j0l7.8160j1j9&sourceid=chrome&ie=UTF-8", 
                 "https://www.google.com/search?q=anti+gun+control+articles&rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk03dy0dQykODeYwIiSbiVrPd3MidIA:1582179712378&ei=gCVOXvTRFtbt-gTlhJWoAw&start=10&sa=N&ved=2ahUKEwj0kejdvt_nAhXWtp4KHWVCBTUQ8tMDegQIEBAu&biw=1766&bih=767"]

accepted_words = ['gun', 'second', 'firearms','shooting','kill']


scraped_links = get_scraped_links(all_gun_links, accepted_words)
get_body_updated_to_df(scraped_links, "guns", "a", 
                       'gun|guns|firearm|amendment|second|control|rifle|shooting|shoot')

In [ ]:
# Immigration

all_imm_links = ["https://www.nytimes.com/topic/subject/illegal-immigration",
                 "https://www.nationalreview.com/immigration/",
                 "https://www.foxnews.com/category/us/immigration",
                 "https://www.politico.com/magazine/tag/immigrants",
                 "https://www.politico.com/magazine/tag/immigration-reform", 
                 "https://www.fairus.org/issues/illegal-immigration", 
                 "https://time.com/tag/immigration/", 
                 "https://www.newyorker.com/tag/illegal-immigrants",
                 "https://www.theguardian.com/us-news/usimmigration", 
                 "https://www.pewresearch.org/fact-tank/2019/06/28/what-we-know-about-illegal-immigration-from-mexico/", 
                 "https://www.cnn.com/specials/politics/immigration", 
                 "https://www.whitehouse.gov/issues/immigration/",
                 "https://www.migrationpolicy.org/topics/illegal-immigration-interior-enforcement",
                 "https://scholar.google.com/scholar?q=illegal+immigration+debate&hl=en&as_sdt=0&as_vis=1&oi=scholart",
                 "https://www.google.com/search?q=illegal+immgration+debate&rlz=1C5CHFA_enUS811US824&oq=illegal+immgration+debate&aqs=chrome..69i57j0j69i60j69i61j69i65j69i60l3.3298j1j4&sourceid=chrome&ie=UTF-8",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk01ln8nOxLRBaEehMoJtUjSX2sBXsA%3A1582180668510&ei=PClOXuTgHsHH-gSCk5TgCw&q=illegal+immigration+programs&oq=illegal+immigration+programs&gs_l=psy-ab.3...60865.62948..63384...1.0..0.86.729.10......0....1..gws-wiz.......35i39j0i131j0j0i22i30j0i8i13i30j0i22i10i30.nJ1EKe3vw5I&ved=0ahUKEwjk7t2lwt_nAhXBo54KHYIJBbwQ4dUDCAs&uact=5", 
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ALeKk00e3YbLMOkXQYoWIlSHQ1VyuJ_KzQ%3A1582180759839&ei=lylOXrDiMszh-gSP-ZiIDg&q=daca&oq=daca&gs_l=psy-ab.3..0i67l2j0i131i67i395j0i273i395l2j0i395j0i20i263i395j0i67i395l2j0i395.16892.17581..17778...0.1..0.83.152.2....3..0....1..gws-wiz.......0i71.JKKaLJM0T3U&ved=0ahUKEwjwiaTRwt_nAhXMsJ4KHY88BuEQ4dUDCAs&uact=5", 
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&biw=1766&bih=767&sxsrf=ALeKk02tu5YbK2iyGckNvYxW00kwFFEhvQ%3A1582180827933&ei=2ylOXp6_OI7b-gSaqZ2IDw&q=illegal+immigration+into+the+united+states&oq=illegal+immigration+into+the+united+states&gs_l=psy-ab.3...5608.9642..9805...0.1..0.86.2665.42....2..0....1..gws-wiz.......0i71j35i39j0i67j0i131j0i395j0i131i395j0j0i131i67j0i273.psEfKpBVO7k&ved=0ahUKEwjemODxwt_nAhWOrZ4KHZpUB_EQ4dUDCAs&uact=5"]

accepted_words = ['alien','immigrant','immigration','daca','dream','deport','border','citizen','citizenship', 'illegal','refugee','mexico','deport','undocumented','detain','detention','unauthorized']
scraped_links = get_scraped_links(all_imm_links, accepted_words)

get_body_updated_to_df(scraped_links, "immigration", "a", 
                       'immigrant|immigration|illegal|citizen|citizenship|border|wall|refugee|sanctuary|undocumented|detain|detention|unauthorized')


In [ ]:
# Healthcare

all_hc_links = ["https://www.nytimes.com/topic/subject/medicare",
                 "https://www.nytimes.com/topic/subject/health-insurance-and-managed-care",
                 "https://www.nytimes.com/topic/subject/health-care-reform",
                 "https://www.cnn.com/specials/politics/health-care",
                 "https://www.economist.com/topics/health-care", 
                 "https://www.statnews.com/category/politics/", 
                 "https://www.modernhealthcare.com/politics-policy", 
                 "https://www.isidewith.com/poll/894292236",
                 "https://www.isidewith.com/poll/965572", 
                 "https://www.nejm.org/medical-research/politics-of-health-care-reform", 
                 "https://www.theguardian.com/us-news/healthcare", 
                 "https://www.mercer.us/our-thinking/healthcare-reform.html",
                 "https://www.cnbc.com/medicare/",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ACYBGNS6bqZShgL7A1HenRy4gAVjtX1qkw%3A1581889327590&ei=L7dJXrrBI9LG-gSD-7SQCA&q=medicare+politics&oq=medicare+politics&gs_l=psy-ab.3..0j0i22i30l2.11243.12557..12730...0.2..0.100.668.8j1......0....1..gws-wiz.......0i71j35i39j0i67.O5pYQ0_xmag&ved=0ahUKEwj6lMf7hNfnAhVSo54KHYM9DYIQ4dUDCAs&uact=5",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ACYBGNTLCwA08H3eLj75KcBQNq1ycp7hbg:1581889341342&q=political+impact+on+medicare&sa=X&ved=2ahUKEwj-0Y6ChdfnAhXP7Z4KHWrnBZUQ1QIoAHoECAsQAQ&biw=1382&bih=599&dpr=2",
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&sxsrf=ACYBGNS5bcrOMbHS_PbMDY6CLIrOZ3U8rA:1582003344154&q=problems+with+medicare&sa=X&ved=2ahUKEwjTjvLardrnAhUjFTQIHU6_CMcQ1QIoBHoECAsQBQ&biw=1382&bih=648", 
                 "https://www.google.com/search?rlz=1C5CHFA_enUS811US824&biw=1382&bih=648&sxsrf=ACYBGNThW9-YAjsYvQaBNDoNiWIP9YizqQ%3A1582003356824&ei=nHRLXsP3Ma7O0PEP4dGQCA&q=medicare-for-all&oq=medicare-for-all&gs_l=psy-ab.3..0l6j0i30l4.45842.49291..49602...0.4..0.164.758.6j3......0....1..gws-wiz.......0i71j35i39j0i67j0i273.xNxLIXGwozM&ved=0ahUKEwjDtffgrdrnAhUuJzQIHeEoBAEQ4dUDCAs&uact=5"]

accepted_words = ['medicare','insurance','healthcare','medicaid','universal','medical','hospital','obamacare','affordable','aca','care','drug','pharma','regulate']
scraped_links = get_scraped_links(all_hc_links, accepted_words)

get_body_updated_to_df(scraped_links, "healthcare", "a", 
                       'medicare|medicaid|medical|insurance|universal|hospital|healthcare|affordable|obamacare|care|act|drug|pharma|regulate')

In [ ]:
# Abortion

all_abortion_links = ["https://www.theatlantic.com/magazine/archive/2019/12/the-things-we-cant-face/600769/",
                     "https://www.bbc.com/news/world-us-canada-47940659",
                     "https://www.nationalreview.com/2019/11/honest-abortion-debate-caitlin-flanagan-article-the-atlantic/",
                     "https://www.gale.com/open-access/abortion",
                     "https://en.wikipedia.org/wiki/Category:Abortion_debate",
                     "https://www.nytimes.com/2019/06/15/us/politics/abortion-debate-pennsylvania.html",
                     "https://www.nytimes.com/2020/02/07/us/politics/abortion-litmus-test-debate.html",
                     "https://www.cnn.com/2020/01/23/politics/abortion-attitude-poll-roe-v-wade-anniversary-trnd/index.html",
                     "https://www.newscientist.com/article-topic/abortion/",
                     "https://www.foxnews.com/category/politics/judiciary/abortion",
                     "http://web.csulb.edu/~cwallis/382/readings/160/marquis.html",
                     "https://www.iep.utm.edu/abortion/",
                     "https://www.theguardian.com/world/abortion",
                     "https://www.usnews.com/news/health-news/articles/2020-02-11/states-weigh-fetal-remains-bills-amid-abortion-debate"]

accepted_words = ['abortion','pregnancy','life','choice','woman','women','mother','baby','fetus','child','roe','fetal']
scraped_links = get_scraped_links(all_abortion_links, accepted_words)

get_body_updated_to_df(scraped_links, "abortion", "a", 
                       'abortion|abort|pregnancy|mother|woman|baby|child|woman|women|life|choice|roe|fetal')

In [ ]:
# Free College

all_college_links = ["https://www.vox.com/2019/6/24/18677785/democrats-free-college-sanders-warren-biden",
                     "https://www.cnbc.com/2019/10/01/56percent-of-young-americans-support-free-public-college-how-it-might-work.html",
                     "https://www.theatlantic.com/education/archive/2019/02/2020-democrats-free-college/583585/",
                     "https://www.insidehighered.com/news/2019/05/03/poll-support-free-college-among-young-people",
                     "https://www.collegeraptor.com/find-colleges/articles/affordability-college-cost/pros-cons-tuition-free-college/",
                     "https://www.nytimes.com/2019/09/20/learning/should-college-be-free.html",
                     "https://smartasset.com/student-loans/the-pros-and-cons-of-free-college",
                     "https://www.prosancons.com/education/pros-and-cons-of-free-education/",
                     "https://www.bebee.com/producer/@gordon-filworth/should-college-be-free-pros-cons-and-alternatives",
                     "https://psmag.com/education/why-are-free-college-programs-so-successful",
                     "https://www.inquirer.com/opinion/commentary/free-college-tuition-2020-candidates-20190905.html",
                     "https://college-education.procon.org/",
                     "https://www.apa.org/pi/ses/resources/indicator/2016/10/free-tuition",
                     "https://www.dissentmagazine.org/article/tressie-mcmillan-cottom-why-free-college-necessary"]

accepted_words = ['college','tuition','debt','free','loan','education','529','afford']
scraped_links = get_scraped_links(all_college_links, accepted_words)

get_body_updated_to_df(scraped_links, "college", "a", 
                       'college|tuition|university|debt|free|loan|education|529|afford')

In [ ]:
# Military Spending

all_military_links = ["https://www.washingtonpost.com/us-policy/2019/04/18/us-military-spending-set-increase-fifth-consecutive-year-nearing-levels-during-height-iraq-war/",
                     "https://www.thebalance.com/u-s-military-budget-components-challenges-growth-3306320",
                     "https://slate.com/news-and-politics/2020/02/trump-military-budget-defense-740-billion-analysis.html",
                     "https://www.militarytimes.com/news/pentagon-congress/2019/07/23/budget-deal-gives-a-big-funding-boost-to-defense-but-republicans-wanted-more/",
                     "https://spacenews.com/trumps-2020-budget-increases-defense-spending-by-5-percent-funds-space-force/",
                     "https://www.charleskochinstitute.org/issue-areas/foreign-policy/the-military-spending-debate/",
                     "https://www.theatlantic.com/ideas/archive/2019/01/democrats-would-be-wrong-cut-us-military-spending/579457/",
                     "https://www.illuminateourworld.org/post/2019-military-budgets-by-nation-the-us-is-still-spending-far-more-than-all-its-competitors-combined?gclid=Cj0KCQjwx7zzBRCcARIsABPRscOSpjRZW58XdG_HVkb4G6W3iZzp0Ebc4S54rZO0BUmwYxNuqLhUjz0aAvlTEALw_wcB",
                     "https://www.downsizinggovernment.org/defense/plan-cut-military-spending",
                     "https://en.wikipedia.org/wiki/Military_budget_of_the_United_States",
                     "https://www.cbo.gov/topics/defense-and-national-security/defense-budget",
                     "https://www.brookings.edu/policy2020/votervital/is-us-defense-spending-too-high-too-low-or-just-right/",
                     "https://www.top10answers.com/web?qo=semQuery&ad=semA&q=american%20military%20budget&o=783747&ag=fw4&an=google_s&rch=us250&gclid=Cj0KCQjwx7zzBRCcARIsABPRscNRtc3G9pdzNBR5aVmToHsV7liUOrjn2U9tgvHZe9vxotppH-YKctwaAjl4EALw_wcB"]

accepted_words = ['service','defense','budget','war','spending','military','army','armed force','troops','defend','security','homeland']
scraped_links = get_scraped_links(all_military_links, accepted_words)

get_body_updated_to_df(scraped_links, "military", "a", 
                       'service|defense|spending|budget|military|army|navy|armed|force|troops|war|defend|homeland|security')

In [ ]:
# Wealth Tax

all_tax_links = ["https://www.allsides.com/allsides-search-results?search_api_views_fulltext=wleath+tax&search=wleath+tax&created=2&submit.x=0&submit.y=0",
                 "https://www.nytimes.com/2020/02/21/us/politics/the-liberal-economists-behind-the-wealth-tax-debate.html",
                 "https://www.forbes.com/sites/taxnotes/2020/02/18/the-wealth-tax-debate/#7db64a9771d4",
                 "https://www.weforum.org/agenda/2019/11/the-great-wealth-tax-debate/",
                 "https://www.cnn.com/2019/02/17/opinions/wealth-tax-is-bad-idea-andelman/index.html",
                 "https://www.politico.com/newsletters/morning-tax/2019/11/21/wealth-tax-back-in-the-debate-782866",
                 "https://www.cnbc.com/2019/11/20/elizabeth-warren-pushes-wealth-tax-during-democratic-debate.html",
                 "https://www.cbsnews.com/news/elizabeth-warren-wealth-tax-who-would-pay-and-how-much/",
                 "https://www.democracynow.org/2020/1/15/democratic_debate_higher_education_universal_programs",
                 "https://www.npr.org/sections/money/2019/12/17/787476334/is-a-wealth-tax-constitutional",
                 "https://taxfoundation.org/wealth-tax/",
                 "https://www.nytimes.com/2019/11/15/your-money/wealth-tax-warren-sanders.html",
                 "https://thehill.com/blogs/congress-blog/politics/474984-a-wealth-tax-is-a-tax-on-business",
                 "https://www.theatlantic.com/technology/archive/2019/11/bill-gates-wrong-about-warrens-wealth-tax/601594/",
                "https://www.wsj.com/articles/wealth-tax-is-a-decent-idea-though-probably-unconstitutional-11575591063",
                "https://www.cnbc.com/2019/12/23/millionaires-support-a-wealth-tax-as-long-as-they-arent-getting-taxed.html"]

accepted_words = ['rich','wealth','middle','class','tax','socialism','socialist','capitalist','capital','raise','trickle','millionaire','billionaire']
scraped_links = get_scraped_links(all_tax_links, accepted_words)

get_body_updated_to_df(scraped_links, "tax", "a", 
                       'tax|social|socialism|socialist|capital|capitalist|raise|middle|wealth|rich|trickle|millionaire|billionaire')

## Step 3: Clean the DataFrame

Now that we have scraped all our data sources, we must combine all the dataframes for each topic into a single dtaframe and deduplicate entries that appear twice.

In [ ]:
def clean_df(subject):
    
    # Read in all files of a given subject. Note that due the above code, all related files have the same naming convention
    cleaned_df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', subject + "*.csv"))))
    
    # Sorting values and counting rows
    cleaned_df = cleaned_df.sort_values(by=['url', 'Unnamed: 0'])
    print('Before dedupe:', cleaned_df.shape)
    
    # Dropping duplicates
    cleaned_df = cleaned_df.drop(['url', 'Unnamed: 0'], axis=1).drop_duplicates()
    print('After dedupe:', cleaned_df.shape)
    
    # Drop extra index column
    cleaned_df = cleaned_df.reset_index(drop=True)
    
    # Write output to a CSV
    cleaned_df.to_csv(subject + "_cleaned.csv")

In [ ]:
clean_df("guns")
clean_df("immigration")
clean_df("healthcare")
clean_df("abortion")
clean_df("college")
clean_df("military")
clean_df("tax")

## Step 4: Consolidate into a single text file for pretraining
The following was run as a short .py file. The contents of the .py file are pasted below for readability

In [ ]:
import pandas as pd 

# Combining all the L2 data into one large CSV
df_guns = pd.read_csv("guns_cleaned.csv")
df_immigration = pd.read_csv("immigration_cleaned.csv")
df_healthcare = pd.read_csv("healthcare_cleaned.csv")
df_abortion = pd.read_csv("abortion_cleaned.csv")
df_college = pd.read_csv("college_cleaned.csv")
df_military = pd.read_csv("military_cleaned.csv")
df_tax = pd.read_csv("tax_cleaned.csv")

df_list = [df_guns, df_immigration, df_healthcare, df_abortion, df_college, df_military, df_tax]

all_strings = []
for d in df_list:
    col = pd.Series(d.text.unique())   
    s = col.str.cat(sep=' ')
    s = s + "\n"
    all_strings.append(s)

# Write CSV
file1 = open("L2_7topics.txt","w") 
  
# \n is placed to indicate EOL (End of Line) 
file1.writelines(all_strings) 
file1.close() #to change file access modes 